
# Proyecto DS – Parte I (Visualizaciones en Python)

## Abstract
Este proyecto utiliza el dataset **Winequality-red.csv** (conjunto clásico del dominio enológico) para explorar la relación entre atributos fisicoquímicos del vino tinto y su **calidad** sensorial (variable objetivo ordinal). El objetivo es aplicar un **Análisis Exploratorio de Datos (EDA)** siguiendo las prácticas vistas en el curso: lectura con **Pandas**, tratamiento básico de **datos ausentes**, generación de **resúmenes numéricos** y **visualizaciones** con **Matplotlib** (univariadas, bivariadas y multivariadas), y contrastar resultados con **preguntas e hipótesis** explícitas.

El dataset contiene mediciones como **acidez volátil**, **ácido cítrico**, **azúcares residuales**, **cloruros**, **dióxido de azufre libre/total**, **densidad**, **pH**, **sulfatos**, **alcohol**, y una calificación de **calidad** asignada por catadores. La **motivación** es entender qué propiedades se asocian con mayores calificaciones y qué patrones emergen en la distribución de los atributos. Desde la perspectiva de **Industria 4.0** y **estrategias data-driven**, esta exploración ilustra cómo los datos permiten **mejorar decisiones** (p. ej., control de proceso) mediante **métricas descriptivas** y visualizaciones adecuadas.

Planteamos hipótesis inspiradas en conocimiento de dominio y criterios analíticos: (H1) **mayor alcohol** se asocia positivamente con **mayor calidad**; (H2) **menor acidez volátil** se asocia con mayor calidad; (H3) **azúcares residuales** muestran **relación débil** con la calidad; (H4) combinaciones multivariadas (alcohol–acidez volátil–sulfatos) podrían explicar mejor la variabilidad que relaciones bivariadas aisladas. Para contrastarlas utilizaremos histogramas, boxplots, diagramas de dispersión y una matriz de correlaciones, junto con **medidas de resumen** (media, mediana, IQR, desviación estándar).

Los resultados esperados son: identificar **tendencias** (p. ej., alcohol ↑ → calidad ↑), **patrones de dispersión** y potenciales **outliers** (a tratar de forma informada), además de cuantificar asociaciones lineales mediante **correlación**. Finalmente, se discuten **implicancias prácticas** (p. ej., qué variables monitorear) y **limitaciones** (posible asimetría, multicolinealidad y naturaleza ordinal de la calidad). Este trabajo sienta las bases para etapas posteriores del ciclo de un **modelo analítico**: selección de variables, validación e incluso **modelos supervisados** en futuras iteraciones.



## Preguntas e hipótesis de interés
**P1.** ¿Cómo se distribuyen los principales atributos fisicoquímicos (alcohol, acidez volátil, pH, sulfatos)?  
**P2.** ¿Existe relación **bivariada** entre *alcohol* y *calidad*? ¿y entre *acidez volátil* y *calidad*?  
**P3.** ¿Una **combinación multivariada** (p. ej., alcohol–acidez volátil–sulfatos) ofrece mejor diagnóstico que mirar variables por separado?  
**P4.** ¿Hay **valores ausentes** o atípicos a considerar para etapas posteriores?  

**Hipótesis:**  
- **H1:** A mayor **alcohol**, mayor **calidad**.  
- **H2:** A menor **acidez volátil**, mayor **calidad**.  
- **H3:** **Azúcares residuales** muestran asociación débil con la calidad.  
- **H4:** El trinomio **alcohol–acidez volátil–sulfatos** resume mejor la variabilidad observada.


In [ ]:
import pandas as pd
CSV_URL = "https://raw.githubusercontent.com/gerardohamburg/CoderHouse/main/winequality-red.csv"
df = pd.read_csv(CSV_URL, sep=';')

In [ ]:

# =========================
# Estructura y valores ausentes
# =========================
print("\nTipos de datos:\n", df.dtypes)
print("\nValores ausentes por columna:\n", df.isnull().sum())

# % de ausentes
na_pct = df.isnull().mean().sort_values(ascending=False) * 100
display(pd.DataFrame({"porc_ausentes": na_pct}).T)


In [ ]:

# =========================
# Resúmenes numéricos
# =========================
desc = df.describe().T
display(desc)

# Medidas adicionales de dispersión y forma si se requieren
iqr = (df.quantile(0.75) - df.quantile(0.25)).rename("IQR")
resume = desc.join(iqr, how="left")
display(resume)



## Visualizaciones univariadas
A continuación, histogramas para variables clave (alcohol, acidez volátil, pH, sulfatos) para responder **P1** y contextualizar **H1** y **H2**.


In [ ]:

# Histograms (Matplotlib, sin estilos específicos)
vars_uni = ["alcohol", "volatile acidity", "pH", "sulphates"]
fig, axes = plt.subplots(2, 2, figsize=(10, 6))
axes = axes.ravel()

for ax, col in zip(axes, vars_uni):
    if col in df.columns:
        ax.hist(df[col].dropna(), bins=30)
        ax.set_title(col)
        ax.set_xlabel(col)
        ax.set_ylabel("Frecuencia")
    else:
        ax.text(0.5, 0.5, f"{col} no encontrado", ha="center")
plt.tight_layout()
plt.show()



**Interpretación (univariado).**  
- **alcohol**: distribuciones con sesgo moderado pueden anticipar relación positiva con **quality** (apoya **H1**).  
- **volatile acidity**: valores más altos suelen percibirse negativamente (posible relación inversa con **quality**, apoya **H2**).  
- **pH** y **sulphates** permiten contexto sobre acidez y estructura; observar asimetrías y rango.



## Visualizaciones bivariadas
Para **P2**, contrastamos **alcohol vs. quality** y **volatile acidity vs. quality** con dispersión y boxplots por categoría de calidad.


In [ ]:

# Dispersión: alcohol vs. quality y volatile acidity vs. quality
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

if "alcohol" in df.columns and "quality" in df.columns:
    axes[0].scatter(df["alcohol"], df["quality"])
    axes[0].set_xlabel("alcohol")
    axes[0].set_ylabel("quality")
    axes[0].set_title("Alcohol vs Quality")

if "volatile acidity" in df.columns and "quality" in df.columns:
    axes[1].scatter(df["volatile acidity"], df["quality"])
    axes[1].set_xlabel("volatile acidity")
    axes[1].set_ylabel("quality")
    axes[1].set_title("Volatile Acidity vs Quality")

plt.tight_layout()
plt.show()

# Boxplots por categoría de calidad
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
if "quality" in df.columns and "alcohol" in df.columns:
    # Preparamos listas por clase de calidad
    groups = [df.loc[df["quality"]==q, "alcohol"].dropna() for q in sorted(df["quality"].unique())]
    axes[0].boxplot(groups, positions=sorted(df["quality"].unique()))
    axes[0].set_xlabel("quality")
    axes[0].set_ylabel("alcohol")
    axes[0].set_title("Alcohol por Quality")

if "quality" in df.columns and "volatile acidity" in df.columns:
    groups2 = [df.loc[df["quality"]==q, "volatile acidity"].dropna() for q in sorted(df["quality"].unique())]
    axes[1].boxplot(groups2, positions=sorted(df["quality"].unique()))
    axes[1].set_xlabel("quality")
    axes[1].set_ylabel("volatile acidity")
    axes[1].set_title("Volatile Acidity por Quality")

plt.tight_layout()
plt.show()



**Interpretación (bivariado).**  
- **H1:** Tendencia creciente de **quality** con **alcohol** respalda la hipótesis si se observa pendiente positiva y medianas más altas en boxplots.  
- **H2:** **volatile acidity** con pendiente negativa y medianas más altas en calidades bajas apoyaría la relación inversa esperada.



## Visualización multivariada (≥3 variables)
Para **P3**, incorporamos una tercera variable (p. ej., **sulphates** como tamaño del marcador) en la relación **alcohol–volatile acidity** para diagnosticar patrones conjuntos.


In [ ]:

# Scatter 2D con tamaño representando sulphates
plt.figure(figsize=(6,4))
if set(["alcohol","volatile acidity","sulphates"]).issubset(df.columns):
    sizes = (df["sulphates"] - df["sulphates"].min() + 1) * 20  # escala simple para tamaño
    plt.scatter(df["alcohol"], df["volatile acidity"], s=sizes)
    plt.xlabel("alcohol")
    plt.ylabel("volatile acidity")
    plt.title("Alcohol vs Volatile Acidity (tamaño ~ sulphates)")
    plt.show()
else:
    print("Columnas requeridas no disponibles para el gráfico multivariado.")



**Interpretación (multivariado).**  
Si los vinos con **alcohol** alto y **acidez volátil** baja presentan **mayores sulphates** (tamaños mayores) y se corresponden con **mejor quality** (contrastable en cortes por calidad), esto sugiere que combinaciones de variables explican mejor la variabilidad (**H4**).



## Matriz de correlaciones
Como complemento, una vista global de correlaciones lineales entre variables numéricas.


In [ ]:

num_df = df.select_dtypes(include=[np.number])
corr = num_df.corr()

plt.figure(figsize=(8,6))
im = plt.imshow(corr, aspect='auto')
plt.colorbar(im, fraction=0.046, pad=0.04)
plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
plt.yticks(range(len(corr.columns)), corr.columns)
plt.title("Matriz de correlaciones (Pearson)")
plt.tight_layout()
plt.show()

# Correlaciones clave con quality
if "quality" in num_df.columns:
    print(corr["quality"].sort_values(ascending=False))



## Valores perdidos (P4) y nota sobre outliers
- **Valores perdidos** ya identificados arriba (tabla de ausentes). Si se detectan, se propone **imputación** simple (media/mediana/moda) o **eliminación** selectiva según impacto y proporción.  
- **Outliers:** se recomienda inspección con **boxplots** (ya incluidos) y, de ser necesario, *winsorización* o transformaciones (p. ej., log) antes de modelado.



## Vinculación de resultados con preguntas/hipótesis
- **P1 / Univariado:** histogramas describen la **distribución** y **dispersión** (media, mediana, IQR), base para entender rangos razonables.  
- **P2 / Bivariado:** dispersión y boxplots permiten evaluar **H1** (alcohol↑ → quality↑) y **H2** (volatile acidity↑ → quality↓).  
- **P3 / Multivariado:** el uso de **tamaño** (sulphates) añade una tercera dimensión que ayuda a interpretar clústeres o gradientes conjuntos (**H4**).  
- **P4 / Calidad de datos:** la tabla de ausentes guía decisiones de **limpieza** antes de pasar a selección de variables o modelos.



## Conclusiones y próximos pasos
El EDA sugiere asociaciones **coherentes** con el conocimiento de dominio: **alcohol** tiende a asociarse positivamente con **quality**, mientras que **acidez volátil** lo haría negativamente. La inspección multivariada indica que combinaciones de atributos (p. ej., **alcohol–acidez volátil–sulphates**) aportan diagnóstico adicional.  
Como siguientes pasos dentro del **modelo analítico**: (i) consolidar tratamiento de **nulos/outliers**, (ii) explorar **correlaciones** para evitar multicolinealidad, (iii) preparar datos (escalado/codificación) y (iv) avanzar a **modelos supervisados** (clasificación/regresión) con **validación cruzada** y métricas adecuadas.
